In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd

# Leapfrog N-Body Code with Python

Newton's Law of Gravity:

$$
\vec{F} = -\frac{m_1 m_2}{r^3}\vec{r},\quad\mathrm{or}\quad \vec{a_1} = \vec{F} / m_1 = -\frac{m_2}{r^3}\vec{r}
$$

where we've set $G = 1$.

Add softening:

$$
\vec{a_1} = -\frac{m_2}{(r^2 + \epsilon^2)^{3/2}}\vec{r}
$$

where $\epsilon$ is a small number to prevent singularities.

In [ ]:
def accel(m: np.ndarray, x: np.ndarray, eps_sq: float) -> np.ndarray:
    """Compute accelerations for harmonic oscillator(s)

    Args:
        m (array of shape (N,)): masses of points
        x (array of shape (N, 3)): positions of points
        eps_sq (float): softening parameter

    Returns:
        a (array of shape (N, 3)): accelerations of points
    """
    n = len(m)  # number of points
    a = np.zeros((n, 3))  # initialize accelerations
    for i in range(n):  # loop over all points...
        for j in range(i + 1, n):  # loop over i, j pairs
            r_vec = x[i] - x[j]
            r_sq = (r_vec**2).sum()
            acc = m[j] * r_vec / (r_sq + eps_sq)**(3/2)
            a[i] += -acc
            a[j] += acc
    return a


In [ ]:
def leapstep(m: np.ndarray, x: np.ndarray, v: np.ndarray, dt: float, eps_sq: float):
    """Take one step using the leapfrog integrator, formulated
    as a mapping from t to t + dt.  WARNING: this integrator is not
    accurate unless the timestep dt is fixed from one call to another.

    Args:
        m (array of shape (N,)): mass of all points
        x (array of shape (N, 3)): coordinate of all points
        v (array of shape (N, 3)): velocities of all points
        dt (float): timestep for integration
        eps_sq (float): softening parameter (epsilon^2)
    """
    v += 0.5 * dt * accel(m, x, eps_sq) # advance vel by half-step
    x += dt * v # advance pos by full-step
    v += 0.5 * dt * accel(m, x, eps_sq) # complete velocity step


In [ ]:
def save_state(m, x, tnow):
    """Save the current state to file.

    Args:
        m (array of shape (N,)): masses of all points
        x (array of shape (N, 3)): positions of all points
        tnow (float): current time, in seconds
    """
    with open("leapint.py.data", "a") as f:
        for i in range(len(m)):  # loop over all points...
            f.write("%8.4f%12.6f%12.6f%12.6f%12.6f\n" % (tnow, m[i], x[i][0], x[i][1], x[i][2]))

In [ ]:
%%time

n = 100 # number of points

m = np.zeros(n)
x = np.zeros((n, 3))
v = np.zeros((n, 3))

# read initial condition file
with open("initc.data", "r") as f:
    content = f.readlines()

# load header information
header = content[0].split()
dt = float(header[2]) # timestep for integration
tmax = float(header[3]) # duration of the simulation
eps_sq = float(header[4]) # softening parameter (epsilon^2)

# load n-body initial conditions
for i in range(n):
    line = content[i + 1].split()
    m[i] = float(line[0])
    x[i, :] = [float(line[1]), float(line[2]), float(line[3])]
    v[i, :] = [float(line[4]), float(line[5]), float(line[6])]

# next, set integration parameters
tnow = 0.0  # set initial time
mstep = int(tmax / dt)  # number of steps to take

# delete old output if it exists
if os.path.exists("leapint.py.data"):
    os.remove("leapint.py.data")

# now, loop performing integration
for nstep in range(mstep):  # loop mstep times in all
    leapstep(m, x, v, dt, eps_sq)  # take integration step
    tnow = tnow + dt  # and update value of time
    save_state(m, x, tnow)  # then output last step

# Plot the Results of the N-body Simulation with Python

In [ ]:
# load data
# sphere.data has lines structured as mass, x, y, z
t, m, x, y, z = [], [], [], [], []
with open('leapint.py.data', "r") as f:
    for line in f.readlines():
        data = line.split()
        t.append(float(data[0]))
        m.append(float(data[1]))
        x.append(float(data[2]))
        y.append(float(data[3]))
        z.append(float(data[4]))

In [ ]:
num_bodies = 100
num_steps = len(m) // num_bodies

# delete old plots
for file in os.listdir('build'):
    if file.startswith('py_'):
        os.remove(os.path.join('build', file))

# save plots of frames
for i in range(num_steps):
    plt.figure(figsize=(10, 10))
    plt.scatter(x[i*num_bodies:(i+1)*num_bodies], y[i*num_bodies:(i+1)*num_bodies], s=1)
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.title(f'time = {t[i*num_bodies]:.2f}')
    plt.savefig(f'build/py_{i:03d}.png')
    plt.close()

In [ ]:
# create video
!ffmpeg -r 30 -i build/py_%03d.png build/py.mp4 -y -loglevel error

In [ ]:
# show video
ipd.Video('build/py.mp4')

# N-Body Simulation with Aarseth's Code

Run 

```
gcc nbody0-lab.c -o nbody0-lab.out -lm
./nbody0-lab.out
```

in the terminal.

# Plot the Results of the N-body Simulation from Aarseth's Code

In [ ]:
# load data
# sphere.data has lines structured as mass, x, y, z
t, m, x, y, z = [], [], [], [], []
with open('sphere.data', "r") as f:
    for line in f.readlines():
        data = line.split()
        t.append(float(data[0]))
        m.append(float(data[1]))
        x.append(float(data[2]))
        y.append(float(data[3]))
        z.append(float(data[4]))

num_bodies = 2000
num_steps = len(m) // num_bodies

# delete old plots
for file in os.listdir('build'):
    if file.startswith('aarseth_'):
        os.remove(os.path.join('build', file))

# save plots of frames
for i in range(num_steps):
    plt.figure(figsize=(10, 10))
    plt.scatter(x[i*num_bodies:(i+1)*num_bodies], y[i*num_bodies:(i+1)*num_bodies], s=1)
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.title(f'time = {t[i*num_bodies]:.2f} s')
    plt.savefig(f'build/aarseth_{i:03d}.png')
    plt.close()

# create video
!ffmpeg -r 30 -i build/aarseth_%03d.png build/aarseth.mp4 -y -loglevel error

# show video
ipd.Video('build/aarseth.mp4')